In [1]:
#%%

# google colab configurations 
# these might change
DATA_GENERATION = '13kwW8C01etWsqTYoE2eeFlD4Ft9YVhrw'
DATA_GENERATION_FILE = 'datageneration.py'

HELPERS = '1zskLM0-ZtEQYVI7zvd2ePqtkTOkJmVIL'
HELPERS_FILE = 'helpers.py'

EXPERIMENT = '1OrYHcTvUXHOrg-86_6oWaN2muDpuzn9a'
EXPERIMENT_FILE = 'experiment.py'

NETWORK = '1iFbCIY8IMaqO4I8mu2J7ie7UDJsLGtRv'
NETWORK_FILE = 'networks.py'

MODEL_CONFIG = '14OF6o1luI8u10Nq96FLBP1M1k-VvQKq-'
MODEL_CONFIG_FILE ='config.json'

HYPER_CONFIG = '1VWlMZnDhE9yhtBi5LrjZoZjTE6nmXsuk'
HYPER_CONFIG_FILE ='hyperparameter_tuning.json'

TUNER = '1I1K9whIpWvzWYjJQjfHLLrE9GXCuRbb1'
TUNER_FILE = 'param_tuner.py'

google_colab = input('Running on Colab?\n')
if google_colab=='yes' : 
    import os
    os.system('pip install ax ax-platform')
    from pydrive.auth import GoogleAuth
    from pydrive.drive import GoogleDrive
    from google.colab import auth
    from oauth2client.client import GoogleCredentials

    auth.authenticate_user()
    gauth = GoogleAuth()
    gauth.credentials = GoogleCredentials.get_application_default()
    drive = GoogleDrive(gauth)

    datageneration = drive.CreateFile({'id':DATA_GENERATION})
    datageneration.GetContentFile(DATA_GENERATION_FILE)
    

    helpers = drive.CreateFile({'id':HELPERS})
    helpers.GetContentFile(HELPERS_FILE)

    network = drive.CreateFile({'id':NETWORK})
    network.GetContentFile(NETWORK_FILE)

    experiment = drive.CreateFile({'id':EXPERIMENT})
    experiment.GetContentFile(EXPERIMENT_FILE)

    model_json = drive.CreateFile({'id':MODEL_CONFIG})
    model_json.GetContentFile(MODEL_CONFIG_FILE)

    hyper_json = drive.CreateFile({'id':HYPER_CONFIG})
    hyper_json.GetContentFile(HYPER_CONFIG_FILE)

    param_tuner = drive.CreateFile({'id':TUNER})
    param_tuner.GetContentFile(TUNER_FILE)
    


Running on Colab?
yes


In [0]:
# custom imports
from datageneration import DataGenerator
from helpers import Helper

# torch imports
from torch.utils import data
import torch

# import utils
import json
from datetime import date
import os
from functools import partial
import numpy as np



#custom imports
from param_tuner import HyperParam

In [0]:
# we first get the variables for the configuration file
os.environ["CHECKPOINT_DIR"] = "checkpoints"
os.environ["MODEL_PARAMS"] = "default"

CHECKPOINT_DIR = os.environ['CHECKPOINT_DIR']
MODEL_PARAMS = os.environ['MODEL_PARAMS']

In [0]:
if google_colab=='yes':
    config_director = ''
else:
    config_director = 'configs/'

with open(f'{config_director}config.json') as f:
    file = f.read()
    default_conf = json.loads(file)

with open(f'{config_director}hyperparameter_tuning.json') as f:
    file = f.read()
    hyper_conf = json.loads(file)

In [0]:
test_params = hyper_conf['one_d_regression']

In [7]:
dtype = torch.float
train_on_gpu = torch.cuda.is_available()
if(train_on_gpu):
    print('Training on GPU!')
    device = torch.device("cuda")

else: 
    print('No GPU available, training on CPU')
    device = torch.device("cpu")
    test_params[-1]['value']=False

Training on GPU!


In [0]:
datagenerator = DataGenerator(**default_conf[MODEL_PARAMS]['data_gen_params'])
x_values, func_x = datagenerator.generate_curves()
func_x = Helper.list_np_to_sensor(func_x)
x_values = x_values.repeat(func_x.shape[0], 1, 1)



train_len = int(x_values.shape[0] * default_conf[MODEL_PARAMS]['train_share'])
traindata = data.TensorDataset(x_values[:train_len], func_x[:train_len])
trainloader = data.DataLoader(traindata, batch_size=100)
validata = data.TensorDataset(x_values[train_len:], func_x[train_len:])
valiloader = data.DataLoader(validata, batch_size=1)

In [0]:
tuner = HyperParam(test_params, 20)

In [12]:
best_parameters, values, experiment, model = tuner.run_experiment(trainloader, valiloader)

[INFO 04-26 15:26:01] ax.modelbridge.dispatch_utils: Using Sobol generation strategy.
[INFO 04-26 15:26:01] ax.service.managed_loop: Started full optimization with 20 steps.
[INFO 04-26 15:26:01] ax.service.managed_loop: Running optimization trial 1...
  0%|          | 17/10000 [00:00<02:00, 83.13it/s]

Mean loss at epoch 0 : 1.614452838897705


 20%|██        | 2011/10000 [00:24<01:36, 82.75it/s]

Mean loss at epoch 2000 : 1.2732746601104736


 40%|████      | 4013/10000 [00:48<01:11, 83.29it/s]

...

 60%|██████    | 6023/10000 [00:49<00:32, 120.98it/s]

Mean loss at epoch 6000 : 1.1535930633544922


 80%|████████  | 8024/10000 [01:06<00:16, 119.73it/s]

Mean loss at epoch 8000 : 1.1322364807128906


100%|██████████| 10000/10000 [01:22<00:00, 120.62it/s]


In [0]:
best_parameters['type'] = '1_d_regression'

In [15]:
date = f'{str(date.today().year)}_{str(date.today().month).zfill(2)}_{date.today().day}'
path_to_results = os.path.join(os.getcwd(),'tuning_results',date)
with open (path_to_results,'w') as file:
    json.dump(test_params, fp)

FileNotFoundError: ignored